In [ ]:
# Model Selection

In [ ]:
# Load the clustered data

In [ ]:
file_path = 'clustered_data.csv'

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
# Define features and target

In [ ]:
features = ['Principal Component 1', 'Principal Component 2', 'Principal Component 3', 'Principal Component 4']

In [ ]:
target = 'energy_level'

In [ ]:
X = df[features]

In [ ]:
y = df[target]

In [ ]:
# Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Function to evaluate models

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):

In [ ]:
    model.fit(X_train, y_train)

In [ ]:
    y_pred = model.predict(X_test)

In [ ]:
    mae = mean_absolute_error(y_test, y_pred)

In [ ]:
    mse = mean_squared_error(y_test, y_pred)

In [ ]:
    r2 = r2_score(y_test, y_pred)

In [ ]:
    return {'MAE': mae, 'MSE': mse, 'R2': r2}

In [ ]:
## Evaluate Models

In [ ]:
# Models to evaluate

In [ ]:
models = {

In [ ]:
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),

In [ ]:
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),

In [ ]:
    'Neural Network': MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=500, random_state=42)

In [ ]:
}

In [ ]:
results = {}

In [ ]:
for name, model in models.items():

In [ ]:
    results[name] = evaluate_model(model, X_train, X_test, y_train, y_test)

In [ ]:
    print(f"{name} - MAE: {results[name]['MAE']}, MSE: {results[name]['MSE']}, R2: {results[name]['R2']}")

In [ ]:
# Cross-validation

In [ ]:
for name, model in models.items():

In [ ]:
    cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

In [ ]:
    print(f"{name} - Cross-Validated R-squared: {cv_scores.mean()}")

In [ ]:
# Save the results

In [ ]:
results_df = pd.DataFrame(results).T

In [ ]:
results_df.to_csv('model_evaluation_results.csv')

In [ ]:
print("Model evaluation results saved to 'model_evaluation_results.csv'")

In [ ]:
## Trying extra Trees Regressor

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Assuming you want to scale the target variable

In [ ]:
scaler_y = StandardScaler()

In [ ]:
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

In [ ]:
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

In [ ]:
def evaluate_model(model, X_train, X_test, y_train_scaled, y_test_scaled, scaler_y=None):

In [ ]:
    model.fit(X_train, y_train_scaled)

In [ ]:
    y_pred_scaled = model.predict(X_test)

In [ ]:
    if scaler_y:

In [ ]:
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

In [ ]:
    else:

In [ ]:
        y_pred = y_pred_scaled

In [ ]:
    mae = mean_absolute_error(y_test, y_pred)

In [ ]:
    mse = mean_squared_error(y_test, y_pred)

In [ ]:
    r2 = r2_score(y_test, y_pred)

In [ ]:
    return mae, mse, r2

In [ ]:
# Usage with Extra Trees Regressor

In [ ]:
et_model = ExtraTreesRegressor(n_estimators=100, random_state=42)

In [ ]:
mae, mse, r2 = evaluate_model(et_model, X_train, X_test, y_train_scaled, y_test_scaled, scaler_y)

In [ ]:
print(f"Extra Trees - MAE: {mae}, MSE: {mse}, R2: {r2}")

In [ ]:
## Bagging Regressor

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
bg_model = BaggingRegressor(base_estimator=RandomForestRegressor(n_estimators=10, random_state=42), n_estimators=10, random_state=42)

In [ ]:
mae, mse, r2 = evaluate_model(bg_model, X_train, X_test, y_train, y_test)

In [ ]:
print(f"Bagging Regressor - MAE: {mae}, MSE: {mse}, R2: {r2}")

In [ ]:
# Cross-validation for Bagging Regressor

In [ ]:
bg_cv_scores = cross_val_score(bg_model, X, y, cv=5, scoring='r2')

In [ ]:
print(f"Bagging Regressor - Cross-Validated R-squared: {bg_cv_scores.mean()}")

In [ ]:
# Model Evaluation

In [ ]:
import numpy as np

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
residuals = y_test - y_pred

In [ ]:
plt.scatter(y_test, residuals)

In [ ]:
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max(), colors='red')

In [ ]:
plt.xlabel('Actual Values')

In [ ]:
plt.ylabel('Residuals')

In [ ]:
plt.title('Residual Plot')

In [ ]:
plt.show()